# Intro to profiling

Python's dirty little secret is that it can be made to run pretty fast.  

The bare-metal HPC people will be angrily tweeting at me now, or rather, they would be if they could get their wireless drivers working.

Still, there are some things you *really* don't want to do in Python.  Nested loops are usually a bad idea.  But often you won't know where your code is slowing down just by looking at it and trying to accelerate everything can be a waste of time.  (Developer time, that is, both now and in the future: you incur technical debt if you unintentionally obfuscate code to make it faster when it doesn't need to be).

The first step is always to find the bottlenecks in your code, via _profiling_: analyzing your code by measuring the execution time of its parts.

Tools
-----

2. `cProfile`
1. [`line_profiler`](https://github.com/rkern/line_profiler)
3. `timeit`

**Note**:
If you haven't already installed it, you can do

```console
conda install line_profiler
```

or

```console
pip install line_profiler
```

## Some bad code


Here's a bit of code guaranteed to perform poorly: it sleeps for 1.5 seconds after doing any work! We will profile it and see where we might be able to help.

In [1]:
import numpy
from time import sleep

def bad_call(dude):
    sleep(.5)
    
def worse_call(dude):
    sleep(1)
    
def sumulate(foo):
    if not isinstance(foo, int):
        return
    
    a = numpy.random.random((1000, 1000))
    a @ a
    
    ans = 0
    for i in range(foo):
        ans += i
        
    bad_call(ans)
    worse_call(ans)
        
    return ans

In [2]:
sumulate(150)

11175

## using `cProfile`

[`cProfile`](https://docs.python.org/3.4/library/profile.html#module-cProfile) is the built-in profiler in Python (available since Python 2.5).  It provides a function-by-function report of execution time. First import the module, then usage is simply a call to `cProfile.run()` with your code as argument. It will print out a list of all the functions that were called, with the number of calls and the time spent in each.

In [3]:
import cProfile

In [4]:
cProfile.run('sumulate(150)')

         10 function calls in 1.572 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.055    0.055    1.571    1.571 <ipython-input-1-5c3dbf5afb25>:10(sumulate)
        1    0.000    0.000    0.501    0.501 <ipython-input-1-5c3dbf5afb25>:4(bad_call)
        1    0.000    0.000    1.001    1.001 <ipython-input-1-5c3dbf5afb25>:7(worse_call)
        1    0.001    0.001    1.571    1.571 <string>:1(<module>)
        1    0.000    0.000    1.572    1.572 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    1.502    0.751    1.502    0.751 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.014    0.014    0.014    0.014 {method 'random_sample' of 'mtrand.RandomState' objects}




You can see here that when our code `sumulate()` executes, it spends almost all its time in the method `time.sleep` (a bit over 1.5 seconds).

If your program is more complicated that this cute demo, you'll have a hard time parsing the long output of `cProfile`. In that case, you may want a profiling visualization tool, like [SnakeViz](https://jiffyclub.github.io/snakeviz/). But that is outside the scope of this tutorial.

## using `line_profiler`

`line_profiler` offers more granular information thatn `cProfile`: it will give timing information about each line of code in a profiled function.

Load the `line_profiler` extension

In [5]:
%load_ext line_profiler

### For a pop-up window with results in notebook:

IPython has an `%lprun` magic to profile specific functions within an executed statement. Usage:
`%lprun -f func_to_profile <statement>` (get more help by running `%lprun?` in IPython).

### Profiling two functions

In [6]:
%lprun -f bad_call -f worse_call sumulate(13)

### Write results to a text file

In [7]:
%lprun -T timings.txt -f sumulate sumulate(12)


*** Profile printout saved to text file 'timings.txt'. 


In [8]:
# %load timings.txt

## Profiling on the command line

Open file, add `@profile` decorator to any function you want to profile, then run

```console
kernprof -l script_to_profile.py
```

which will generate `script_to_profile.py.lprof` (pickled result).  To view the results, run

```console
python -m line_profiler script_to_profile.py.lprof
```

In [9]:
from IPython.display import IFrame

In [15]:
# IFrame('http://localhost:7000/terminals/1', width=800, height=700)

## `timeit`

`timeit` is not perfect, but it is helpful.  

Potential concerns re: `timeit`

* Only runs benchmark 3 times
* It disables garbage collection

```python
python -m timeit -r 25 "print(42)"
```

```python
python -m timeit -s "gc.enable()" "print(42)"
```

Victor Stinner has a module, `perf`, which is more robust and addresses these concerns. You can check it out at: https://perf.readthedocs.io/en/latest/user_guide.html

### Line magic

In [11]:
%timeit x = 5

14 ns ± 0.998 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


### Cell magic

In [12]:
%%timeit
x = 5
y = 6
x + y

37.6 ns ± 0.534 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


The `-q` flag quiets output.  The `-o` flag allows outputting results to a variable.  The `-q` flag sometimes disagrees with OSX so please remove it if you're having issues.

In [13]:
a = %timeit -qo x = 5